In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
import pandas as pd
import numpy as np

# own Modules 
from models_sub_net_ls import LstmMse_LatentSpace, LstmMle_LatentSpace, AnalysisLayer
from models_mse import LstmMse
from models_mle import LstmMle_1
from data_preperator import DataPreperatorPrediction
from data_set import DataSet
from predictor import PredictorMse, PredictorMle, PredictorMseLatentSpaceAnalyser, PredictorMleLatentSpaceAnalyser

## Take care of these things before training:
- Select correct path and define droped_features
- Change parameter of model
- Change filed_location

## Parameters phm data

In [2]:
param = {
    "data" : {
        "path" : '../../data/phm_data_challenge/01_M01_DC_prediction_1.csv',
        "droped_feature" : ["stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_model/phm_dataFold xx_InputSize13_LayerLstm1_HiddenLstm15_HiddenFc75_Seq25.pt",
        "input_size" : 13,
        "n_hidden_lstm" : 15,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "n_hidden_fc_pred": 75,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE/phm_mle_1.csv",
    }
}

## Parameters artifical data

In [16]:
param = {
    "data" : {
        "path" : '../../data/artifical_signals/MLE_analysis/artifical_2_signals_errors.csv',
        "droped_feature" : ["anomaly"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_latent_space/artifical_2_signals_InputSize2_LayerLstm1_HiddenLstm15_HiddenFc_pred75_HiddenFc_ls7_Seq25.pt",
        "input_size" : 2,
        "n_hidden_lstm" : 15,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "n_hidden_fc_pred": 75,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE_LS/artifical_2_signals.csv",
    }
}

## Parameters cpps data

In [2]:
param = {
    "data" : {
        "path" : '../../data/cpps_data/cpps_data_predictive_maintenance.csv',
        "droped_feature" : ["status"
                           ],
        "features_not_to_scale": []
    },
    "model" : {
        "path" : "../../models/MLE_latent_space/cpps_data_InputSize10_LayerLstm1_HiddenLstm15_HiddenFc_pred75_HiddenFc_ls7_Seq25.pt",
        "input_size" : 10,
        "n_hidden_lstm" : 15,
        "sequence_size" : 100,
        "batch_size" : 50,
        "lstm_layer" : 1,
        "n_hidden_fc_pred": 75,
        "n_hidden_fc_ls": 7,
        "dropout_rate_lstm": 0.0,
        "dropout_rate_fc": 0.2,
        "K":1
    },
        "anomaly_detection" : {
        "threshold_anomaly" : 0.3,
        "smooth_rate" : 0.05,
        "no_standard_deviation" : 2
    },
    "results": {
        "path" : "../visualisation/files/prediction/MLE_LS/cpps_data.csv",
    }
}

## Standarize Data
First we have to apply normalisation to data. That is because the model works on the representation given by its input vectors. The scale of those numbers is part of the representation.
We should apply the exact same scaling as for training data. That means storing the scale and offset used with your training data, and using that again. <br>
__The mean and variance for each feature of the training data with which the model was trained (stake: 0.75):__

### Mean and Variance from phm Dataset
droped features="stage", "Lot", "runnum", "recipe", "recipe_step",
                            "up time", "ongoing time", 
                            "ETCHSOURCEUSAGE", "ETCHAUXSOURCETIMER", 
                            "ETCHAUX2SOURCETIMER", "FIXTURESHUTTERPOSITION"

In [55]:
mean_training_data =[ 0.21683119,  0.32121513,  0.31925213,  0.20097501,  0.45164471,  0.22914814,
  0.11604865,  0.27421592,  0.24393222, -0.13974937, -0.09739598, -0.07313758,
  0.18198089]
var_training_data =[0.75261122, 0.90482986, 0.91105327, 0.75504036, 1.07026701, 0.76708319,
 0.35172769, 0.83004988, 0.76964675, 0.57386915, 0.45912309, 0.2955709,
 1.61493449]

### Mean and Variance from artifical Dataset

In [7]:
mean_training_data= [-0.00393712, -0.01294209]
var_training_data = [49.18936568,  0.34270256]

### Mean and Variance form cpps Dataset

In [19]:
mean_training_data = [0.05330162, 0.03075699, -0.05636937, 0.0274802, 0.06536314, -0.04620979,-0.0745559, 
                      -0.08149049, -0.05318843, 0.11105582]
var_training_data = [0.02905961, 0.04473883, 0.05254194, 0.05198144, 0.07337494, 0.0666981, 0.07593811, 
                     0.0393896, 0.08028017, 0.0594492]

## Create DataLoader

In [20]:
data_preperator = DataPreperatorPrediction(path=param['data']['path'], 
                                           ignored_features = param["data"]["droped_feature"],
                                           mean_training_data=mean_training_data, 
                                           var_training_data=var_training_data, 
                                           first_order_difference=False 
                                          )                                  
preprocessed_data = data_preperator.prepare_data()
print(preprocessed_data.shape)

dataset = DataSet(preprocessed_data, 
                  timesteps=param["model"]["sequence_size"])
data_loader = DataLoader(dataset, 
                         batch_size=param['model']['batch_size'], 
                         num_workers=0, 
                         shuffle=False, 
                         drop_last=True)

(150000, 11)


In [21]:
for batch_idx, data in enumerate(data_loader):
    x,y = data
    print('Data of batch: {}'.format(batch_idx))
    print("Size of input data: {}".format(x.size()))
    print("Size of target data: {}".format(y.size()))
    if batch_idx >=1: break

Data of batch: 0
Size of input data: torch.Size([50, 100, 11])
Size of target data: torch.Size([50, 11])
Data of batch: 1
Size of input data: torch.Size([50, 100, 11])
Size of target data: torch.Size([50, 11])


## Define Model and load Parameters of trained model
### Model for MSE and Latent Space Analysis

In [ ]:
model = LstmMse_LatentSpace(batch_size=param['model']['batch_size'], 
                            input_dim=param['model']['input_size'], 
                            n_hidden_lstm=param['model']['n_hidden_lstm'], 
                            n_layers=param['model']['lstm_layer'],
                            dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                            dropout_rate_fc= param['model']['dropout_rate_fc'],
                            n_hidden_fc_prediction=param['model']['n_hidden_fc_pred'], 
                            n_hidden_fc_ls_analysis=param['model']['n_hidden_fc_ls']      
                            )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

### Model for MLE and Latent Space Analysis

In [3]:
model = LstmMle_LatentSpace(batch_size=param['model']['batch_size'], 
                            input_dim=param['model']['input_size'], 
                            n_hidden_lstm=param['model']['n_hidden_lstm'], 
                            n_layers=param['model']['lstm_layer'],
                            dropout_rate_lstm= param['model']['dropout_rate_lstm'],
                            dropout_rate_fc= param['model']['dropout_rate_fc'],
                            n_hidden_fc_prediction=param['model']['n_hidden_fc_pred'], 
                            n_hidden_fc_ls_analysis=param['model']['n_hidden_fc_ls'],
                            K = param['model']['K']
                            )

checkpoint = torch.load(param["model"]["path"])
model.load_state_dict(checkpoint['model_state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Initialize Predictor
### Predictor for MSE Model and Latent Space Analysis

In [ ]:
predictor = PredictorMseLatentSpaceAnalyser(model=model,
                                            path_data=param["data"]["path"],
                                            columns_to_ignore=param["data"]["droped_feature"],
                                            threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"]
                                            )

### Predictor for MLE Model and Latent Space Analysis

In [4]:
predictor = PredictorMleLatentSpaceAnalyser(model=model,
                                            path_data=param["data"]["path"],
                                            columns_to_ignore=param["data"]["droped_feature"],
                                            threshold_anomaly=param["anomaly_detection"]["threshold_anomaly"],
                                            no_standard_deviation=param["anomaly_detection"]["no_standard_deviation"]
                                            )

## Predict

In [25]:
print("Start predicting.")
# Write header
with open(param["results"]["path"], "a+") as file:
            [file.write(column+";") for column in predictor.create_column_names_result()]
            file.write("\n")

for batch_number, (input_data, target_data) in enumerate(data_loader):
    # Predict sensor values in mini-batches
    batch_results = predictor.predict(input_data, target_data)
    
    # Write results to csv file
    with open(param["results"]["path"], "a") as file:
        for batch in batch_results:
            # Each result component of a singe prediction (ID, target, prediction, loss, latent space ...) is stored in lists
            # thus we have to unpack the list and seperate values with ;
            for value in batch:
                file.write(str(value)+";")
            file.write("\n")

    # Print status 
    if (batch_number*param['model']['batch_size'])%5000 == 0:
        print("Current status: " + str(param['model']['batch_size']*batch_number) + " samples are predicted.")

print("End of prediction.")

Start predicting.
Current status: 0 samples are predicted.
Current status: 5000 samples are predicted.
Current status: 10000 samples are predicted.
Current status: 15000 samples are predicted.
Current status: 20000 samples are predicted.
Current status: 25000 samples are predicted.
Current status: 30000 samples are predicted.
Current status: 35000 samples are predicted.
Current status: 40000 samples are predicted.
Current status: 45000 samples are predicted.
Current status: 50000 samples are predicted.
Current status: 55000 samples are predicted.
Current status: 60000 samples are predicted.
Current status: 65000 samples are predicted.
Current status: 70000 samples are predicted.
Current status: 75000 samples are predicted.
Current status: 80000 samples are predicted.
Current status: 85000 samples are predicted.
Current status: 90000 samples are predicted.
Current status: 95000 samples are predicted.
Current status: 100000 samples are predicted.
Current status: 105000 samples are predic

## Tag anomalous samples

In [5]:
results_prediction = pd.read_csv(param["results"]["path"], sep=";")
# Values of column "loss" are exponentially smoothed and stored in a new column "smoothed loss"
# New column "anomaly" is created and sample is taged with 1 if anomalous behaviour (if smoothed loss is over threshold)
results = predictor.detect_anomaly(results_prediction, param["anomaly_detection"]["smooth_rate"])

In [6]:
results.head()

,ID,signal_1 target,signal_2 target,signal_3 target,signal_4 target,signal_5 target,signal_6 target,signal_7 target,signal_8 target,signal_9 target,...,Anomaly Sensor_1,Anomaly Sensor_2,Anomaly Sensor_3,Anomaly Sensor_4,Anomaly Sensor_5,Anomaly Sensor_6,Anomaly Sensor_7,Anomaly Sensor_8,Anomaly Sensor_9,Anomaly Sensor_10
0,101.0,-0.892629,-1.455301,-1.159004,0.665475,0.270881,-1.595616,0.630266,0.166484,0.557938,...,0,0,0,0,0,0,0,0,0,0
1,102.0,-0.915688,-1.418095,-1.213050,0.677133,0.190997,-1.575921,0.572661,0.126792,0.511696,...,0,0,0,0,0,0,0,0,0,0
2,103.0,-0.953605,-1.338766,-1.305150,0.696980,0.055676,-1.532497,0.464926,0.049242,0.428981,...,0,0,0,0,0,0,0,0,0,0
3,104.0,-1.002367,-1.228261,-1.438691,0.734270,-0.131096,-1.478106,0.318320,-0.056791,0.319786,...,0,0,0,0,0,0,0,0,0,0
4,105.0,-1.060904,-1.090143,-1.593820,0.774040,-0.351596,-1.406483,0.139286,-0.187528,0.186531,...,0,0,0,0,0,0,0,0,0,0


## Combine prediction data with data which was not consider for inference

In [7]:
original_sensor_data = pd.read_csv(param["data"]["path"])
data_of_droped_feature = original_sensor_data.loc[:, param["data"]["droped_feature"]+["ID"]]
complete_data = results.merge(right=data_of_droped_feature, how="inner", on="ID")

## Save data to csv file

In [8]:
complete_data.to_csv(param["results"]["path"], sep=";", index=False)